In [1]:
import pandas as pd
import numpy as np
import os

os.chdir('/home/kruu/git_folder/eye_tracking/')
from utils.data_processing import EyeTrackingProcessor

import torch
from torch.utils.data import DataLoader
from models.lstm_classifier import LSTMClassifier, train_classifier, evaluate_model
from utils.dataset import GazeMouseDataset

****
# Import Datasets
****

In [ ]:
data_path = "/store/kruu/eye_tracking"
files_list = os.listdir(data_path)
files_list = [os.path.join(data_path, file) for file in files_list]

tasks = ['Task 1', 'Task 2', 'Task 3', 'Task 4', 'Task 5', 'Task 6']
features = ['Recording timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y', 'Participant name']
interpolate_col = ['Recording timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y']

processor = EyeTrackingProcessor()
all_data = processor.load_data(files_list)
dataset = processor.get_features(all_data, tasks, features)

#Fixed size resampling for the whole task
features = ["Gaze point X", "Gaze point Y", "Mouse position X", "Mouse position Y", "Recording timestamp"]
dataset_resampled = processor.fixed_points_resample(dataset, features, num_points=2000)
dataset_resampled.fillna(0, inplace=True)

#Fixed Time step resampling
# dataset_resampled = processor.fixed_time_steps_resample(dataset, interpolate_col, timestep = 0.01)
# dataset_resampled.fillna(0, inplace=True)

/home/kruu/git_folder/eye_tracking/utils/data_processing.py:20: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep='\t')
/home/kruu/git_folder/eye_tracking/utils/data_processing.py:20: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep='\t')


In [11]:
dataset_resampled

,Resampled Index,Gaze point X,Gaze point Y,Mouse position X,Mouse position Y,Recording timestamp,Participant name,Task_id,Task_execution
0,0.000000,1670.000000,884.000000,-918.000000,312.000000,2.083195e+08,1,1,0
1,0.010101,1093.000000,1055.000000,-442.030303,556.333333,2.087037e+08,1,1,0
2,0.020202,812.000000,1057.000000,-292.515152,643.696970,2.091320e+08,1,1,0
3,0.030303,794.000000,1061.000000,-181.727273,911.545455,2.095278e+08,1,1,0
4,0.040404,808.212121,1065.787879,614.000000,943.000000,2.099447e+08,1,1,0
...,...,...,...,...,...,...,...,...,...
43195,0.959596,774.000000,717.000000,975.000000,451.000000,8.269190e+08,12,6,5
43196,0.969697,800.363636,782.181818,975.000000,451.000000,8.271206e+08,12,6,5
43197,0.979798,626.000000,1353.000000,975.000000,451.000000,8.273334e+08,12,6,5
43198,0.989899,626.000000,1353.000000,975.000000,451.000000,8.275461e+08,12,6,5


****
# Classifier
****

In [ ]:
features = ['Relative timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y']
model = train_classifier(dataset_resampled,
                          features,
                          batch_size=64,
                          hidden_dim=64,
                          num_layers=4,
                          learning_rate=0.001,
                          num_epochs=100)

****
# Analysis
****

In [4]:
ckpt_path = "/home/kruu/git_folder/eye_tracking/GazeMouse_Classification/2ajznad2/checkpoints/best_lstm_classifier.ckpt"
features = ['Relative timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y']
    
model_load = LSTMClassifier(input_dim=5, hidden_dim=64, num_classes = 6, num_layers=4, learning_rate=0.001)
model_load.load_state_dict(torch.load(ckpt_path)["state_dict"])
model_load.eval()

LSTMClassifier(
  (lstm): LSTM(5, 64, num_layers=4, batch_first=True)
  (fc): Linear(in_features=64, out_features=6, bias=True)
  (criterion): CrossEntropyLoss()
)

In [5]:
res = evaluate_model(model_load, dataset_resampled, features)

✅ Evaluation Complete: Loss = 3.0381, Accuracy = 0.1852
